In [1]:
import json

import numpy as np
import pandas as pd

from collections import defaultdict

# 1. Results

## 1.1 KNN

## 1.1.1 BERT

In [2]:
# reading KNN-BERT results
with open('output/results/knn/bert_results.json', 'r') as f:
    result = json.loads(f.read())

In [3]:
# computing the averages
bert_results = defaultdict(lambda: defaultdict(float))
for ki in result.keys():
    for kj in result[ki].keys():
        bert_results[ki][kj] = np.mean(result[ki][kj])

In [4]:
# weighting f1 micro and f1 macro
bert_results = pd.DataFrame(bert_results).T.reset_index()
bert_results['weighted'] = bert_results.micro * bert_results.macro
bert_results = bert_results.sort_values('weighted', ascending=False)
bert_results.head()

,index,exec_time,micro,macro,weighted
20,8 - uniform - cosine,0.486794,0.720933,0.332710,0.239862
28,10 - uniform - cosine,0.473595,0.724000,0.327346,0.236999
24,9 - uniform - cosine,0.484678,0.721894,0.326840,0.235943
36,12 - uniform - cosine,0.471226,0.727504,0.323873,0.235619
44,14 - uniform - cosine,0.466985,0.729608,0.319086,0.232808


The selected configuration is:

- n_neighbors = 8
- weights = uniform
- metric = cosine

## 1.1.2 TF-IDF

In [5]:
# reading KNN-BERT results
with open('output/results/knn/tf_idf_results.json', 'r') as f:
    result = json.loads(f.read())

In [6]:
# computing the averages
tfidf_results = defaultdict(lambda: defaultdict(float))
for ki in result.keys():
    for kj in result[ki].keys():
        tfidf_results[ki][kj] = np.mean(result[ki][kj])

In [7]:
# weighting f1 micro and f1 macro
tfidf_results = pd.DataFrame(tfidf_results).T.reset_index()
tfidf_results['weighted'] = tfidf_results.micro * tfidf_results.macro
tfidf_results = tfidf_results.sort_values('weighted', ascending=False)
tfidf_results.head()

,index,exec_time,micro,macro,weighted
33,11 - uniform - euclidean,5.135127,0.673882,0.305530,0.205891
32,11 - uniform - cosine,5.791443,0.675283,0.300964,0.203236
41,13 - uniform - euclidean,5.182990,0.678437,0.296908,0.201433
45,14 - uniform - euclidean,5.085086,0.679313,0.296208,0.201218
20,8 - uniform - cosine,6.158333,0.668710,0.300681,0.201068


The selected configuration is:

- n_neighbors = 11
- weights = uniform
- metric = euclidean